# Приложение 1: 11. Приложение 2 пункт 2 вариант 4

In [7]:
import numpy as np
from numpy import inf

In [119]:
from math import exp, cos, sqrt
from copy import copy, deepcopy

def argmax(arr): # Первый максимальный по модулю элемент
    abs_arr = [abs(x) for x in arr]
    return abs_arr.index(max(abs_arr))

def str_from_matrix(matrix): # служебная функция
    a = ['\t'.join(list(map(lambda x: str(x),row))) for row in matrix]
    return '\n'.join(a)

def mat_norm(matrix): # вычисление матричной нормы (бесконечная норма)
    return max(map(lambda x: sum(map(abs, x)), matrix))

def print_np_stats(slau):
    print('Определитель: ', np.linalg.det(slau.A))
    print('Число обусловленности: ', np.linalg.cond(slau.A, p=inf))
    print('Корни: ', np.linalg.solve(slau.A,slau.b))
#     print('Обратная матрица ', '\n', np.linalg.inv(slau.A))

def generate_slau(M, n, x): # СЛАУ по формуле из приложения 2 п.2 в. 4
    A = [[0]*n for x in range(n)]
    b = [0]*n
    q = 1.001 - 2*M*pow(10,-3)
    for i in range(1, n+1):
        for j in range(1, n+1):
            if i != j:
                A[i-1][j-1] = pow(q, i+j)+0.1*(j-i)
            else:
                A[i-1][j-1] = pow(q-1, i+j)
        b[i-1] = n*exp(x/i)*cos(x)
    S = Slau(n)
    S.A = A
    S.b = b
    return S

def error(sl, x): # вычисление верхней границы погрешности
    A = sl.A
    f = sl.b
    n = sl.n
    err = [0]*n
    for i in range(n):
        for j in range(n):
            err[i] += A[i][j]*x[j]
        err[i] -= f[i]
    return norm(err)*mat_norm(sl.inv)

def norm(x): # евклидова норма вектора
    return sqrt(sum(map(lambda x: x**2, x)))

def print_tex_matrix(a):
    for i in range(len(a)):
        for j in range(len(a)):
            if j != len(a)-1:
                print('%.4f &' % a[i][j], end='')
            else:
                print('%.4f \\\\' % a[i][j])

def research(sl, epsilon, step = 0.5, start = 0, stop = 2):
    my_range = [x*step for x in range(int(start/step) + 1, int(stop/step))]
    
    for omega in my_range:
        cur, count = sl.over_relax(omega, epsilon)
#         print('Omega = ',round(omega, 2), ' Iter count: ', count)
        print(round(omega,3), ' & ', count, ' \\\\')
                
class Slau: # класс СЛАУ
    def __init__(self, a, y=0): # Инициализация СЛАУ
        if type(a) == str: # Через текстовый файл
            filename = a
            with open(filename, 'r') as f:
                n = int(f.readline())
                A = [[int(x) for x in f.readline().split()] for r in range(n)]
                b = [int(x) for x in f.read().split()]
                self.n = n
            self.A = A
            self.b = b
            self.n = len(A)
        elif type(a) == Slau: # Копия существующей СЛАУ
            matrix = a
            self.A = matrix.copy_A()
            self.b = matrix.copy_b()
            self.n = matrix.n
        elif type(a) == int: # Нулевая СЛАУ
            n = a
            self.A = [[0]*n for row in range(n)]
            self.b = [0]*n
            self.n = n
        elif type(a) == list: # Через матрицу A и вектор столбец y
            self.A = deepcopy(a)
            self.b = copy(y)
            self.n = len(a)
        self.init_solution()
        
    
    def init_solution(self):
        self.x = [0]*self.n # решение СЛАУ
        self.transposition = list(range(self.n)) # перестановки столбцов
        self.k = 0 # количество перестановок
        self.C = self.copy_A() # треугольная матрциа после метода Гаусса
        self.y = self.copy_b() # правая часть после метода Гаусса
        self.det = 1 # инициализация определителя
    
    def copy_A(self): # Копия матрицы коэффициентов
        return [list(a) for a in self.A]
    
    def copy_b(self): # Копия вектора y
        return list(self.b)
    
    def __str__(self): # Представление матрицы в текстовом виде 
        return str_from_matrix(self.A)
    
    def swap_col(self, a, b, matrix): # Поменять столбцы местами
        if matrix == self.C: # учёт перестановки решений
            if a != b:
                self.k+=1
            ind_a = self.transposition.index(a)
            ind_b = self.transposition.index(b)
            self.transposition[ind_a] = b
            self.transposition[ind_b] = a
        for i in range(self.n):
            tmp = matrix[i][a]
            matrix[i][a] = matrix[i][b]
            matrix[i][b] = tmp
    
    def gauss_straight(self, modified = 0): # Прямой ход метода Гаусса
        for i in range(self.n):
            if modified: # Модифицированный метод Гаусса
                ind = argmax(self.C[i]) # поиск максимального элемента в строке
                self.swap_col(i, ind, self.C) # перестановка столбцов
            else: # Обычный метод Гаусса
                for x in range(i+1, self.n): # поиск строки с ненулевым iым элементом и перестановка строк
                    if self.C[x][i] != 0:
                        tmp = self.C[i]
                        self.C[i] = self.C[x]
                        self.C[x] = tmp
                        tmp = self.y[i]
                        self.y[i] = self.y[x]
                        self.y[x] = tmp
                        if i != x:
                            self.k+=1
                        break
            el = self.C[i][i]
            if el == 0:
                print('Вырожденная матрица')
                return -1
            
            for j in range(self.n):
                self.C[i][j] /= el
            
            self.y[i] = self.y[i] / el
            self.det *= el # подсчёт определителя
            
            for j in range(i+1, self.n):
                first = self.C[j][i]
                for k in range(i, self.n):
                    self.C[j][k] -= self.C[i][k]*first
                self.y[j] -= self.y[i]*first
    
    def gauss_reverse(self): # Обратный ход метода Гаусса
        for i in reversed(range(self.n)):
            self.x[i] = self.y[i]
            for j in range(i+1, self.n):
                self.x[i] -= self.x[j]*self.C[i][j]
    
    def solve(self, modified = 0): # Решить СЛАУ
        self.init_solution()
        self.gauss_straight(modified)
        self.gauss_reverse()
        self.det_c() # досчитать определитель после метода Гаусса
        old_x = list(self.x)
        for i in range(self.n): # учесть перестановку неизвестных
            self.x[i] = old_x[self.transposition[i]]
        
        return self.x
    
    def det_c(self): # досчитать определитель 
        self.det *= pow(-1, self.k)
        
    def inverse(self): # Вычисление обратной матрицы(приведение левой(A) части к диагональному виду, правой части(I) к обратной матрице)
        self.inv = [[0]*self.n for row in range(self.n)] # инициализация обратной матрицы
        for i in range(self.n):
            self.inv[i][i] = 1
        
        tmp = self.copy_A()
        for i in range(self.n): # Приведение левой части присоединенной матрицы к верхнетреугольному виду по методу Гаусса 
            for x in range(i+1, self.n): # поиск строки с ненулевым iым элементом и перестановка строк
                if tmp[x][i] != 0: # перестановка строк местами
                    cpy = tmp[i]
                    tmp[i] = tmp[x]
                    tmp[x] = cpy
                    cpy = self.inv[i]
                    self.inv[i] = self.inv[x]
                    self.inv[x] = cpy
                    break
            el = tmp[i][i]
            
            for j in range(self.n):
                tmp[i][j] /= el
                self.inv[i][j] /= el
            
            for j in range(i+1, self.n):
                first = tmp[j][i]
                for k in range(self.n):
                    tmp[j][k] -= tmp[i][k]*first
                    self.inv[j][k] -= self.inv[i][k]*first
        for i in reversed(range(self.n)): # Приведение левой части матрицы к диагональному виду 
            el = tmp[i][i]
            
            for j in reversed(range(0, i)):
                first = tmp[j][i]
                for k in range(0, self.n):
                    tmp[j][k] -= tmp[i][k]*first
                    self.inv[j][k] -= self.inv[i][k]*first
        return self.inv
    
    def condition_number(self): # число обусловленности
        self.inverse()
        return mat_norm(self.A)*mat_norm(self.inv)
    
    def print_stats(self, solve_method = 0): # вывести базовую информацию по СЛАУ
        self.solve(solve_method)
        print('Определитель: %.4f' % self.det)
        print('Число обусловленности: %.4f' % self.condition_number())
        print('Корни: ', self.x)
#         print('Обратная матрица')
#         print_tex_matrix(self.inv)
    
    def over_relax(self, omega, epsilon, max_iter = 20000): # метод верхней релаксации
        transposed = np.array(self.A).T
        A = np.dot(transposed, self.A) # привожу матрицу к симметричному виду
        f = np.dot(transposed, self.b)
        sl = Slau(list(A), f)
        sl.inverse() # посчитать обратную для AA^T матрицу
        
        cur = [0]*self.n
        count = 0
        while error(sl,cur) > epsilon and count < max_iter:
#         while count < max_iter:
            prev = np.copy(cur)
            for i in range(self.n):
                s1 = 0
                for j in range(0, i):
                    s1 += A[i][j]*cur[j]
                s2 = 0
                for j in range(i, self.n):
                    s2 += A[i][j]*prev[j]
                cur[i] = prev[i] + ((f[i] - s1 - s2)*omega)/A[i][i]
            count += 1
        return cur, count
                
        

In [151]:
A = Slau('11-1.txt') 
B = Slau('11-2.txt') # Матрица 11-2 вырожденная, поэтому я поменял в ней элементы a_13 и a_14
C = Slau('11-3.txt') # Матрица 11-3 вырожденная, поэтому я в первой строке переместил a11 в конец, остальные элементы строки сдвинулись влево. Аналогично поступил для 3ей строки
R = Slau('random.txt')
S = Slau('symmetric.txt')
M = 4
n = 100
x = 1
G = generate_slau(M, n, x)
Tests = [A,B,C,R,G]
A = G

In [140]:
# print(A)
# print(A.b)

In [141]:
A.print_stats(0)
print()
print_np_stats(A)

Определитель: 0.2932
Число обусловленности: 332.0582
Корни:  [-28.846110626956744, -19.320904452555666, -14.124976306982726, -9.588915922127025, -5.20740995283289, -0.8346776572999262, 3.5878652933731234, 8.089137237739, 12.685726120796735, 17.388362645893885]

Определитель:  0.29318277478900706
Число обусловленности:  332.0582132255792
Корни:  [-28.84611063 -19.32090445 -14.12497631  -9.58891592  -5.20740995
  -0.83467766   3.58786529   8.08913724  12.68572612  17.38836265]


In [142]:
# x = A.over_relax(omega=1.99, epsilon=0.01, max_iter = 20000)
# print(x)
research(A, epsilon = 0.1, step = 0.5)

0.5  &  20000  \\
1.0  &  9060  \\
1.5  &  2487  \\


In [152]:
A.over_relax(omega=1.529, epsilon=0.1)

([-86.67606587010069,
  -28.540498964873965,
  -14.632159825159459,
  -7.8796256426569835,
  -2.2901909413055583,
  4.8594821390276115,
  14.470360616216107,
  24.542996307524025,
  30.607723645203077,
  29.16576002696537,
  21.34015330936406,
  12.268034931276551,
  6.4299095417823215,
  4.308588635563204,
  3.733567354308668,
  3.0833677751997755,
  2.284127002394127,
  1.698163911701265,
  1.3267834935406868,
  1.016036693093006,
  0.7327070942342242,
  0.49998334336717404,
  0.30827243153515277,
  0.14006551307580364,
  -0.00783475960805826,
  -0.13679488445595103,
  -0.2510027596640202,
  -0.35307861992774653,
  -0.4442964872151458,
  -0.5260500660702496,
  -0.5994621727721059,
  -0.6652359835976814,
  -0.7239292227565886,
  -0.775960080558109,
  -0.8215880553975586,
  -0.8609730833625492,
  -0.8941965088692818,
  -0.921273441292382,
  -0.9421778440290436,
  -0.9568618591225305,
  -0.9652744367848206,
  -0.96738215019027,
  -0.963188834008402,
  -0.9527544861020275,
  -0.936212666

In [150]:
research(A, epsilon = 0.1, step = 0.001, start = 1.52, stop = 1.53)

1.521  &  1795  \\
1.522  &  1764  \\
1.523  &  1735  \\
1.524  &  1707  \\
1.525  &  1680  \\
1.526  &  1655  \\
1.527  &  1631  \\
1.528  &  1608  \\
1.529  &  1588  \\


# Тесты

In [ ]:
print(A)
print(A.b)

In [ ]:
print(np.array(A.A))

In [311]:
print(np.linalg.solve(A.A, A.b))
print(np.linalg.det(A.A))

[-5.  -5.   4.  -0.5]
-23.999999999999993


In [ ]:
print(A.solve(0))
print(A.det)

In [ ]:
print(str_from_matrix(A.C))
print(A.y)

In [ ]:
print(norm(A.A))
print(np.linalg.norm(A.A, ord=inf))

In [ ]:
cond_number = A.condition_number()
print(cond_number)
print(np.linalg.cond(A.A, p=inf))

In [ ]:
print(np.array(A.inv))
print(np.linalg.inv(A.A))